<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-10 10:25:05
-------------------
qualified stocks: 92
with latest results: 30
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.01 L
Current:  1.36 C
-------------------
Today PnL: 74.11 K (0.55%)
Current PnL: -14.12 L (-9.84%)
CY Booked + Current PnL: -8.03 L (-5.6%)
-------------------
Total profit:  4.10 L
Total loss:  -18.22 L
-------------------
Total Booked + Current PnL: 18.95 L (16.26%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.48%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.96 L (53.07%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LICI,785.05,983.0,-2.96,H-LC,89.13,238204.0,43512.0,5574.0,-0.25,22.35,2.34,25.21,34.0,7.81,1.77,31.72,XY25,NTT,INSURANCE
83,UJJIVANSFB,52.77,53.0,57.53,M-SC,86.96,136431.0,-6048.0,6671.0,4.04,-4.24,4.89,0.44,241.0,-0.91,1.01,61.03,OX40N,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.69,M-LC,57.61,105123.0,26761.0,7275.0,0.92,34.15,6.92,43.43,77.0,3.68,0.78,40.60,XY24,NTT,MISC
61,RELIANCE,1291.83,1526.0,6.50,X-LC,26.09,157941.0,17132.0,8387.0,0.01,12.17,5.31,18.13,19.0,2.04,1.17,24.71,XY25,NTT,REFINERIES
77,TATAELXSI,7332.28,7332.0,-10.27,X-MC,65.22,94150.0,-8502.0,8502.0,1.41,-8.28,9.03,-0.00,55.0,-1.00,0.70,41.52,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,41.80,M-SC,43.48,81463.0,127.0,72013.0,0.86,0.16,88.40,88.69,227.0,0.00,0.61,37.37,XR,NTT,CHEMICALS
17,CAMPUS,294.86,393.00,NaN,NaN,85.87,166205.0,-1570.0,57407.0,-0.17,-0.94,34.54,33.28,200.0,-0.03,1.23,28.68,XY24,NTT,FOOTWEAR
21,COALINDIA,406.65,512.15,6.87,L-LC,32.61,149350.0,-1110.0,40145.0,0.66,-0.74,26.88,25.94,157.0,-0.03,1.11,13.96,XY25,ATH,MINING
34,HINDALCO,651.95,761.55,-12.54,M-LC,27.17,105224.0,912.0,16625.0,1.05,0.87,15.80,16.81,71.0,0.05,0.78,17.06,X5K,ATH,METALS
20,CIPLA,1495.00,1795.00,-17.32,H-LC,20.65,100956.0,791.0,19313.0,0.01,0.79,19.13,20.07,30.0,0.04,0.75,8.82,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,JPPOWER,18.73,26.20,-33.95,L-SC,92.39,134269.0,-7573.0,64140.0,9.17,-5.34,47.77,39.88,256.0,-0.12,1.00,36.07,XY24,NTT,POWER
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.30,69120.0,-44429.0,84478.0,8.86,-39.13,122.22,35.27,260.0,-0.53,0.51,69.17,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.00,57.53,M-SC,86.96,136431.0,-6048.0,6671.0,4.04,-4.24,4.89,0.44,241.0,-0.91,1.01,61.03,OX40N,NTT,BANKS
8,ASIANTILES,75.41,137.00,7100.00,M-SC,100.00,78982.0,-14828.0,91445.0,3.64,-15.81,115.78,81.67,235.0,-0.16,0.59,52.95,XR,NTT,CERAMICS
55,NATIONALUM,189.63,247.44,-35.66,L-SC,78.26,102391.0,1887.0,28751.0,3.18,1.88,28.08,30.49,246.0,0.07,0.76,34.89,X5K,ATH,METALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RBLBANK,230.61,388.00,47.23,M-SC,61.96,228752.0,-5548.0,165456.0,-2.23,-2.37,72.33,68.25,242.0,-0.03,1.70,50.64,XY24,NTT,BANKS
19,CERA,8421.60,8422.00,-12.65,X-SC,46.74,90148.0,-19333.0,19337.0,-1.67,-17.66,21.45,0.00,64.0,-1.00,0.67,36.44,OX40N,NTT,CERAMICS
38,ICICIGI,1839.64,2260.25,-10.41,H-MC,58.70,147519.0,11386.0,19738.0,-1.38,8.36,13.38,22.86,33.0,0.58,1.10,22.03,X40,ATH,INSURANCE
11,BANDHANBNK,214.25,400.00,-28.43,H-SC,71.74,213254.0,-35276.0,250744.0,-1.29,-14.19,117.58,86.70,129.0,-0.14,1.58,41.08,XY24,NTT,BANKS
72,SONACOMS,578.05,1006.00,-30.24,M-MC,19.57,90125.0,-11034.0,85925.0,-1.22,-10.91,95.34,74.03,196.0,-0.13,0.67,26.66,AR,BTT,AUTO


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,57.53,M-SC,86.96,136431.0,-6048.0,6671.0,4.04,-4.24,4.89,0.44,241.0,-0.91,1.01,61.03,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-10.27,X-MC,65.22,94150.0,-8502.0,8502.0,1.41,-8.28,9.03,-0.00,55.0,-1.00,0.70,41.52,OX40N,NTT,IT
75,SYMPHONY,1306.42,1306.0,-26.71,M-SC,3.26,156086.0,-15055.0,15000.0,1.03,-8.80,9.61,-0.03,189.0,-1.00,1.16,12.18,OX40N,NTT,DURABLES
48,KANSAINER,299.63,340.0,-67.12,H-SC,11.96,228510.0,-41157.0,77488.0,1.52,-15.26,33.91,13.47,143.0,-0.53,1.70,14.37,XY24,NTT,PAINTS
19,CERA,8421.60,8422.0,-12.65,X-SC,46.74,90148.0,-19333.0,19337.0,-1.67,-17.66,21.45,0.00,64.0,-1.00,0.67,36.44,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-17.44,X-LC,7.61,269521.0,-26413.0,120314.0,0.98,-8.93,44.64,31.73,1.0,-0.22,2.00,7.88,X40,BTT,IT
43,INFY,1461.46,2275.00,-11.70,X-LC,16.30,283787.0,22186.0,123447.0,0.79,8.48,43.50,55.67,2.0,0.18,2.11,15.10,X40,BTT,IT
52,LTIM,5564.16,7197.33,-0.17,X-LC,59.78,192240.0,-8070.0,66861.0,1.56,-4.03,34.78,29.35,3.0,-0.12,1.43,32.81,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.78,X-LC,6.52,151579.0,-49807.0,137422.0,0.08,-24.73,90.66,43.51,5.0,-0.36,1.13,5.51,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.01,X-LC,22.83,267120.0,-3227.0,37851.0,-0.34,-1.19,14.17,12.81,6.0,-0.09,1.98,10.55,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.0,-0.31,M-LC,93.48,175814.0,34974.0,9512.0,-0.49,24.83,5.41,31.59,88.0,3.68,1.31,52.72,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.69,M-LC,57.61,105123.0,26761.0,7275.0,0.92,34.15,6.92,43.43,77.0,3.68,0.78,40.60,XY24,NTT,MISC
51,LICI,785.05,983.0,-2.96,H-LC,89.13,238204.0,43512.0,5574.0,-0.25,22.35,2.34,25.21,34.0,7.81,1.77,31.72,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SAIL,130.64,228.00,103.46,M-MC,77.17,231643.0,6681.0,160969.0,0.52,2.97,69.49,74.53,183.0,0.04,1.72,34.49,XY24,BTT,STEEL
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107170.0,4210.0,41775.0,2.07,4.09,38.98,44.67,68.0,0.10,0.80,38.30,X5K,ATH,METALS
1,AARTIIND,487.04,919.00,41.80,M-SC,43.48,81463.0,127.0,72013.0,0.86,0.16,88.40,88.69,227.0,0.00,0.61,37.37,XR,NTT,CHEMICALS
18,CAMS,3643.00,5250.99,7.56,H-SC,88.04,120988.0,18984.0,26037.0,1.16,18.61,21.52,44.14,125.0,0.73,0.90,37.87,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,57.61,105123.0,26761.0,7275.0,0.92,34.15,6.92,43.43,77.0,3.68,0.78,40.60,XY24,NTT,MISC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-25.79,M-MC,83.70,198064.0,18256.0,94160.0,-0.41,10.15,47.54,62.52,168.0,0.19,1.47,42.53,XR,NTT,BANKS
90,WHIRLPOOL,1167.49,2270.00,-32.39,M-SC,48.91,197127.0,25506.0,136570.0,-1.01,14.86,69.28,94.43,191.0,0.19,1.46,45.49,XR,NTT,DURABLES
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107170.0,4210.0,41775.0,2.07,4.09,38.98,44.67,68.0,0.10,0.80,38.30,X5K,ATH,METALS
55,NATIONALUM,189.63,247.44,-35.66,L-SC,78.26,102391.0,1887.0,28751.0,3.18,1.88,28.08,30.49,246.0,0.07,0.76,34.89,X5K,ATH,METALS
40,INDIAMART,2327.09,4871.06,-49.79,H-SC,56.52,130645.0,7309.0,127523.0,-0.29,5.93,97.61,109.32,122.0,0.06,0.97,29.07,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.30,69120.0,-44429.0,84478.0,8.86,-39.13,122.22,35.27,260.0,-0.53,0.51,69.17,XR,NTT,HOTELS
5,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,84.78,86520.0,-14443.0,109880.0,-0.02,-14.31,127.00,94.53,59.0,-0.13,0.64,28.27,SR,ATH,CHEMICALS
53,MASFIN,326.60,399.50,-20.05,H-SC,73.91,90840.0,-7140.0,29014.0,0.30,-7.29,31.94,22.32,138.0,-0.25,0.67,31.75,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,57.61,105123.0,26761.0,7275.0,0.92,34.15,6.92,43.43,77.0,3.68,0.78,40.60,XY24,NTT,MISC
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107170.0,4210.0,41775.0,2.07,4.09,38.98,44.67,68.0,0.10,0.80,38.30,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ASIANTILES,75.41,137.00,7100.00,M-SC,100.00,78982.0,-14828.0,91445.0,3.64,-15.81,115.78,81.67,235.0,-0.16,0.59,52.95,XR,NTT,CERAMICS
86,VALIANTORG,512.64,838.00,-374.49,H-SC,98.91,116779.0,-16507.0,101096.0,2.37,-12.38,86.57,63.47,144.0,-0.16,0.87,95.75,XR,NTT,CHEMICALS
80,TITAGARH,1097.23,1548.00,-4.54,H-SC,97.83,167650.0,-24365.0,103256.0,0.42,-12.69,61.59,41.08,152.0,-0.24,1.25,41.89,XY24,NTT,ENGINEERING
76,TANLA,917.30,1963.11,-32.58,H-SC,96.74,164991.0,-74424.0,347372.0,0.81,-31.09,210.54,114.01,127.0,-0.21,1.23,51.32,AR,ATH,IT
73,STARHEALTH,551.56,761.00,2.46,M-SC,95.65,167506.0,-32710.0,108728.0,-1.13,-16.34,64.91,37.97,213.0,-0.30,1.24,34.49,XY24,NTT,INSURANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.37
1,25,43.12
2,50,71.79


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.38
XY25     15.12
XR       11.34
X40      10.73
X40N      8.33
OX40N     8.31
AR        7.88
X5K       2.34
X200      1.43
SR        1.15
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    18.45
X-LC    17.80
M-SC    16.39
M-LC    10.00
H-MC     8.85
H-LC     7.77
X-MC     6.14
M-MC     5.82
L-SC     3.19
X-SC     2.73
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.91,-1.03,34.63
IT,10.95,-12.79,70.07
BANKS,10.26,-4.98,52.03
MISC,6.43,-25.20,74.16
FINANCE,6.33,-17.94,63.32
PAINTS,5.57,-16.64,39.62
INSURANCE,5.19,4.48,24.06
ELECTRICAL,5.10,-2.51,46.97
HEALTHCARE,4.65,-2.91,32.97


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2831878.0,26
XR,1184963.0,14
AR,1012065.0,9
X40,606655.0,8
XY25,493871.0,10
X40N,410372.0,8
OX40N,312410.0,11
SR,189457.0,2
X5K,87151.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1886104.0,18
M-SC,1666860.0,20
X-LC,817133.0,11
M-MC,560732.0,5
H-MC,535729.0,7
M-LC,369442.0,8
X-MC,318272.0,5
L-SC,299038.0,5
X-SC,264209.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,882805.0,6
M-SC,XY24,735382.0,6
H-SC,AR,533446.0,3
X-LC,X40,461912.0,4
M-SC,XR,414975.0,5
M-MC,XY24,380647.0,3
H-MC,XY24,221318.0,2
H-SC,XR,203773.0,3
M-LC,XY24,196515.0,4


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
